# 0. Setup

In [1]:
from kiwipiepy import Kiwi
kiwi = Kiwi()

import pandas as pd
from collections import Counter

import json
import os

import ast

### 한국어 일상대화 코퍼스 읽어오기

In [2]:
root_dir = 'corpus'  # corpus 아래 sub-directory에 코퍼스 데이터가 json으로 저장돼있음

records = []

for subdir, dirs, files in os.walk(root_dir):
    for file in files:
        if file.endswith('.json'):
            file_path = os.path.join(subdir, file)
            
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    if os.path.getsize(file_path) > 0:
                        data = json.load(f)
                        
                        for entry in data['info']:
                            for annotation in entry['annotations']['lines']:
                                norm_text = annotation.get('norm_text', '')
                                sex = annotation.get('speaker', {}).get('sex', '')
                                age = annotation.get('speaker', {}).get('age', '')
                                speech_act = annotation.get('speechAct', '')

                                records.append({
                                    'norm_text': norm_text,
                                    'sex': sex,
                                    'age': age,
                                    'speechAct': speech_act
                                })
                    else:
                        print(f"Empty JSON file skipped: {file_path}")
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON in file: {file_path} - {e}")
            except Exception as e:
                print(f"Error processing file: {file_path} - {e}")
                
corpus = pd.DataFrame(records)
corpus.to_csv('corpus.csv', index=False)
corpus

Error decoding JSON in file: corpus/TL_01. KAKAO(1)/KAKAO_1648_13.json - Expecting value: line 137 column 25 (char 4950)


,norm_text,sex,age,speechAct
0,다들 비를 싫어하는 이유는 뭐야 ?,여성,20대,(지시) 질문하기
1,나는 비 좋아 냄새가,여성,20대,(단언) 주장하기
2,나도 비 좋아하는데 그 소리가,남성,30대,(단언) 주장하기
3,비가 오면 습하고 불쾌지수가 높아져,여성,20대,(단언) 주장하기
4,맞아 빨래도 안 말라서 걱정,여성,20대,(단언) 주장하기
...,...,...,...,...
1445938,래브라도 리트리버도 정말 귀엽더라,여성,30대,(단언) 진술하기
1445939,밀란아~ 하고 이름 부르면 절대 안오거든?,여성,30대,(단언) 진술하기
1445940,이름 불러도 안온다고?,여성,30대,(지시) 질문하기
1445941,응 그러다가 아! 맛있다~ 이러면 와,여성,30대,(단언) 진술하기


In [3]:
# kiwi tokenizer를 활용하여, 텍스트의 토큰을 리턴하는 함수

def get_tokens(text):
    tokens = kiwi.tokenize(text)
    tag_pairs = [(token.form, token.tag) for token in tokens]
    return tag_pairs

In [4]:
# 토큰화한 결과 넣어 저장

corpus['tokens'] = corpus['norm_text'].apply(get_tokens)

In [6]:
corpus

,norm_text,sex,age,speechAct,tokens
0,다들 비를 싫어하는 이유는 뭐야 ?,여성,20대,(지시) 질문하기,"[(다, MAG), (들, XSN), (비, NNG), (를, JKO), (싫어하,..."
1,나는 비 좋아 냄새가,여성,20대,(단언) 주장하기,"[(나, NP), (는, JX), (비, NNG), (좋, VA), (어, EC),..."
2,나도 비 좋아하는데 그 소리가,남성,30대,(단언) 주장하기,"[(나, NP), (도, JX), (비, NNG), (좋, VA), (어, EC),..."
3,비가 오면 습하고 불쾌지수가 높아져,여성,20대,(단언) 주장하기,"[(비, NNG), (가, JKS), (오, VV), (면, EC), (습, NNG..."
4,맞아 빨래도 안 말라서 걱정,여성,20대,(단언) 주장하기,"[(맞, VV), (어, EC), (빨래, NNG), (도, JX), (안, MAG..."
...,...,...,...,...,...
1445938,래브라도 리트리버도 정말 귀엽더라,여성,30대,(단언) 진술하기,"[(래브라도, NNP), (리트리버, NNP), (도, JX), (정말, MAG),..."
1445939,밀란아~ 하고 이름 부르면 절대 안오거든?,여성,30대,(단언) 진술하기,"[(밀란, NNP), (아, JKV), (~, SO), (하, VV), (고, EC..."
1445940,이름 불러도 안온다고?,여성,30대,(지시) 질문하기,"[(이름, NNG), (부르, VV), (어도, EC), (안, MAG), (오, ..."
1445941,응 그러다가 아! 맛있다~ 이러면 와,여성,30대,(단언) 진술하기,"[(응, IC), (그러, VV), (다가, EC), (아, IC), (!, SF)..."


In [7]:
corpus.to_csv('tokenized_corpus.csv', index=False)

### 상징 데이터 읽어오기

In [10]:
symbols = pd.read_csv('symbols.csv')

In [11]:
symbols.head()

,Symbol
0,73
1,실수
2,따끔거려요
3,교통사고가-났어요
4,정리해주세요


In [12]:
# '-'는 띄어쓰기를 대신하여 들어가있는 경우가 많음

symbols = symbols.applymap(lambda x: x.replace('-', ' '))
symbols.head()

,Symbol
0,73
1,실수
2,따끔거려요
3,교통사고가 났어요
4,정리해주세요


In [13]:
# 띄어쓰기 적용

symbols = symbols.applymap(lambda x: kiwi.space(x, reset_whitespace=True))
symbols.head()

,Symbol
0,73
1,실수
2,따끔거려요
3,교통사고가 났어요
4,정리해 주세요


In [14]:
symbols['Tokens'] = symbols['Symbol'].apply(get_tokens)
symbols.head()

,Symbol,Tokens
0,73,"[(73, SN)]"
1,실수,"[(실수, NNG)]"
2,따끔거려요,"[(따끔거리, VV), (어요, EF)]"
3,교통사고가 났어요,"[(교통사고, NNG), (가, JKS), (나, VV), (었, EP), (어요,..."
4,정리해 주세요,"[(정리, NNG), (하, XSV), (어, EC), (주, VX), (세요, EF)]"


In [15]:
symbols.to_csv('tokenized_symbols.csv', index=False)

# 1. 문장과 단어 분리

### 1-1.문장 상징 추출

In [16]:
# EF(종결어미)를 갖고 있는 상징은 문장이 될 수 있는 후보들

has_EF = symbols[symbols['Tokens'].apply(lambda tokens: any(pos == 'EF' for _, pos in tokens))]

In [17]:
# EF 갖고 있으면서 token이 4개 이상인 것들은 --> 문장

sentences = has_EF[has_EF['Tokens'].apply(lambda tokens: len(tokens) > 3)]

In [18]:
# EF 갖고 있으면서 token이 3개인 것 중, second last token의 tag가 'XSA', 'XSV'로 시작하지 않으면 --> 문장

sent_append = has_EF[has_EF['Tokens'].apply(lambda tokens: len(tokens) == 3
                                               and not tokens[-2][1].startswith(('XSA', 'XSV')))]

In [19]:
sentences = pd.concat([sentences, sent_append], ignore_index=True)

In [20]:
# 조사를 갖고 있는 것은 문장으로 취급

word_candid = symbols[~symbols['Symbol'].isin(sentences['Symbol'])]
sent_append2 = word_candid[word_candid['Tokens'].apply(lambda tokens: any(pos.startswith('J') for _, pos in tokens))]

In [22]:
sentences = pd.concat([sentences, sent_append2], ignore_index=True)

In [23]:
sentences.head()

,Symbol,Tokens
0,교통사고가 났어요,"[(교통사고, NNG), (가, JKS), (나, VV), (었, EP), (어요,..."
1,정리해 주세요,"[(정리, NNG), (하, XSV), (어, EC), (주, VX), (세요, EF)]"
2,쉬고 싶어요,"[(쉬, VV), (고, EC), (싶, VX), (어요, EF)]"
3,접수는 어떻게 하나요,"[(접수, NNG), (는, JX), (어떻, VA-I), (게, EC), (하, ..."
4,단추를 끼워요,"[(단추, NNG), (를, JKO), (끼우, VV), (어요, EF)]"


In [24]:
len(sentences)

1820

In [20]:
sentences.to_csv('tokenized_symbols_sentence.csv', index=False)

### 1-2. 단어 추출

In [25]:
words = symbols[~symbols['Symbol'].isin(sentences['Symbol'])]

In [26]:
len(words)

5450

In [23]:
words.to_csv('tokenized_symbols_word.csv', index=False)

# 2. 단어 상징의 형태소 통계

In [30]:
# tag(NNG, EF, ...)와 그에 대한 설명 (일반명사, 종결어미, ...)가 담긴 파일

tags = pd.read_csv('korean_pos_tags.csv')

In [40]:
tags_mapping = tags.set_index('Tag')['Description'].to_dict()

In [41]:
word_pos = [token[1] for token_list in words['Tokens'] for token in token_list]

In [42]:
word_pos_cnt = Counter(word_pos)

In [43]:
word_pos_cnt = pd.DataFrame(word_pos_cnt.items(), columns=['Tag', 'Count'])
word_pos_cnt = word_pos_cnt.sort_values(by='Count', ascending=False)
word_pos_cnt.reset_index(drop=True, inplace=True)

word_pos_cnt['Description'] = word_pos_cnt['Tag'].map(tags_mapping)
word_pos_cnt

,Tag,Count,Description
0,NNG,5166,일반 명사
1,EF,779,종결 어미
2,NNP,563,고유 명사
3,VV,387,동사
4,SN,223,숫자(0-9)
5,VA,143,형용사
6,XSV,137,동사 파생 접미사
7,XSA,111,형용사 파생 접미사
8,SSO,105,SS 중 여는 부호*
9,SSC,103,SS 중 닫는 부호*


In [54]:
# 단독 형식 형태소로 존재하는 상징이 있는가?

def has_empty_morph(tokens):
    if len(tokens)==1:
        pos = tokens[0][1]
        if pos in ('VX', 'VCP', 'VCN'): #보조용언, 이다, 아니다
            print(tokens[0])
        elif pos.startswith('J'): #조사
            print(tokens[0])
        elif pos.startswith('E'): #어미
            print(tokens[0])
        elif pos.startswith('XS' or 'XP'): #접두사
            print(tokens[0])
                
_ = symbols['Tokens'].apply(has_empty_morph)

('라지', 'EF')


### --> 형식 형태소를 나타내는 상징은 없음. '라지(large)'는 잘못 분석된 것.   
### 따라서 단어 상징과 자연어 코퍼스를 비교할 때, 둘 다 형태소분석을 한 후 실질 형태소만을 뽑아와서 비교할 것

In [55]:
# (참고용) 자연어 코퍼스의 형태소 통계

corpus_pos = [token[1] for token_list in corpus['tokens'] for token in token_list]
corpus_pos_cnt = Counter(corpus_pos)

corpus_pos_cnt = pd.DataFrame(corpus_pos_cnt.items(), columns=['Tag', 'Count'])
corpus_pos_cnt = corpus_pos_cnt.sort_values(by='Count', ascending=False)
corpus_pos_cnt.reset_index(drop=True, inplace=True)

corpus_pos_cnt['Description'] = corpus_pos_cnt['Tag'].map(tags_mapping)

corpus_pos_cnt

,Tag,Count,Description
0,NNG,2650143,일반 명사
1,EC,1444338,연결 어미
2,VV,1441058,동사
3,EF,1230417,종결 어미
4,MAG,971689,일반 부사
5,JX,742332,보조사
6,SF,665579,종결 부호(. ! ?)
7,ETM,587347,관형형 전성 어미
8,VA,562083,형용사
9,NNB,538521,의존 명사


In [56]:
def get_lexical_morph(tokens):
    result = []
    if type(tokens)==str:
        tokens = ast.literal_eval(tokens)
    for token in tokens:
        pos = token[1]
        if pos.startswith('N'):
            result.append(token)
        elif pos.startswith('V') and pos not in ('VX', 'VCP', 'VCN'):
            result.append(token)
        elif pos in ('MM', 'MAG', 'MAJ', 'IC', 'XR'):
            result.append(token)
    
    return result

In [57]:
words['Lexical_morph'] = words['Tokens'].apply(get_lexical_morph)

/var/folders/jg/85wzy6zn3f9gvrwtm5c2jn2w0000gn/T/ipykernel_20285/1769120092.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  words['Lexical_morph'] = words['Tokens'].apply(get_lexical_morph)


In [58]:
words

,Symbol,Tokens,Lexical_morph
0,73,"[(73, SN)]",[]
1,실수,"[(실수, NNG)]","[(실수, NNG)]"
2,따끔거려요,"[(따끔거리, VV), (어요, EF)]","[(따끔거리, VV)]"
6,핸드 컨트롤러,"[(핸드, NNG), (컨트롤러, NNG)]","[(핸드, NNG), (컨트롤러, NNG)]"
7,날개,"[(날개, NNG)]","[(날개, NNG)]"
...,...,...,...
7264,고리 끼우기,"[(고리, NNG), (끼우, VV), (기, ETN)]","[(고리, NNG), (끼우, VV)]"
7265,동식물,"[(동식물, NNG)]","[(동식물, NNG)]"
7266,복어,"[(복어, NNG)]","[(복어, NNG)]"
7267,평면도형,"[(평면도, NNG), (형, XSN)]","[(평면도, NNG)]"


In [35]:
words.to_csv('tokenized_symbols_lex_morph.csv', index=False)

# 3. 자연어 형태소 set에서 상징 형태소 set이 포함된 비율

In [59]:
symbol_set = set().union(*words['Lexical_morph'])

In [60]:
len(symbol_set)

4705

In [61]:
corpus

,norm_text,sex,age,speechAct,tokens
0,다들 비를 싫어하는 이유는 뭐야 ?,여성,20대,(지시) 질문하기,"[(다, MAG), (들, XSN), (비, NNG), (를, JKO), (싫어하,..."
1,나는 비 좋아 냄새가,여성,20대,(단언) 주장하기,"[(나, NP), (는, JX), (비, NNG), (좋, VA), (어, EC),..."
2,나도 비 좋아하는데 그 소리가,남성,30대,(단언) 주장하기,"[(나, NP), (도, JX), (비, NNG), (좋, VA), (어, EC),..."
3,비가 오면 습하고 불쾌지수가 높아져,여성,20대,(단언) 주장하기,"[(비, NNG), (가, JKS), (오, VV), (면, EC), (습, NNG..."
4,맞아 빨래도 안 말라서 걱정,여성,20대,(단언) 주장하기,"[(맞, VV), (어, EC), (빨래, NNG), (도, JX), (안, MAG..."
...,...,...,...,...,...
1445938,래브라도 리트리버도 정말 귀엽더라,여성,30대,(단언) 진술하기,"[(래브라도, NNP), (리트리버, NNP), (도, JX), (정말, MAG),..."
1445939,밀란아~ 하고 이름 부르면 절대 안오거든?,여성,30대,(단언) 진술하기,"[(밀란, NNP), (아, JKV), (~, SO), (하, VV), (고, EC..."
1445940,이름 불러도 안온다고?,여성,30대,(지시) 질문하기,"[(이름, NNG), (부르, VV), (어도, EC), (안, MAG), (오, ..."
1445941,응 그러다가 아! 맛있다~ 이러면 와,여성,30대,(단언) 진술하기,"[(응, IC), (그러, VV), (다가, EC), (아, IC), (!, SF)..."


In [62]:
corpus['lexical_morph'] = corpus['tokens'].apply(get_lexical_morph)
corpus

,norm_text,sex,age,speechAct,tokens,lexical_morph
0,다들 비를 싫어하는 이유는 뭐야 ?,여성,20대,(지시) 질문하기,"[(다, MAG), (들, XSN), (비, NNG), (를, JKO), (싫어하,...","[(다, MAG), (비, NNG), (싫어하, VV), (이유, NNG), (뭐,..."
1,나는 비 좋아 냄새가,여성,20대,(단언) 주장하기,"[(나, NP), (는, JX), (비, NNG), (좋, VA), (어, EC),...","[(나, NP), (비, NNG), (좋, VA), (냄새, NNG)]"
2,나도 비 좋아하는데 그 소리가,남성,30대,(단언) 주장하기,"[(나, NP), (도, JX), (비, NNG), (좋, VA), (어, EC),...","[(나, NP), (비, NNG), (좋, VA), (그, MM), (소리, NNG)]"
3,비가 오면 습하고 불쾌지수가 높아져,여성,20대,(단언) 주장하기,"[(비, NNG), (가, JKS), (오, VV), (면, EC), (습, NNG...","[(비, NNG), (오, VV), (습, NNG), (불쾌지수, NNG), (높아..."
4,맞아 빨래도 안 말라서 걱정,여성,20대,(단언) 주장하기,"[(맞, VV), (어, EC), (빨래, NNG), (도, JX), (안, MAG...","[(맞, VV), (빨래, NNG), (안, MAG), (마르, VV), (걱정, ..."
...,...,...,...,...,...,...
1445938,래브라도 리트리버도 정말 귀엽더라,여성,30대,(단언) 진술하기,"[(래브라도, NNP), (리트리버, NNP), (도, JX), (정말, MAG),...","[(래브라도, NNP), (리트리버, NNP), (정말, MAG), (귀엽, VA-I)]"
1445939,밀란아~ 하고 이름 부르면 절대 안오거든?,여성,30대,(단언) 진술하기,"[(밀란, NNP), (아, JKV), (~, SO), (하, VV), (고, EC...","[(밀란, NNP), (하, VV), (이름, NNG), (부르, VV), (절대,..."
1445940,이름 불러도 안온다고?,여성,30대,(지시) 질문하기,"[(이름, NNG), (부르, VV), (어도, EC), (안, MAG), (오, ...","[(이름, NNG), (부르, VV), (안, MAG), (오, VV)]"
1445941,응 그러다가 아! 맛있다~ 이러면 와,여성,30대,(단언) 진술하기,"[(응, IC), (그러, VV), (다가, EC), (아, IC), (!, SF)...","[(응, IC), (그러, VV), (아, IC), (맛있, VA), (이렇, VA..."


In [63]:
corpus_set = set().union(*corpus['lexical_morph'])

In [64]:
len(corpus_set)

74675

In [65]:
symbol_in_corpus = corpus_set.intersection(symbol_set)

In [66]:
len(symbol_in_corpus)

4377

In [67]:
len(symbol_in_corpus) / len(corpus_set)

0.05861399397388684

# 4. 상징이 포함된 문장은 전체 문장의 몇 %?

In [68]:
# count how many elements of the list are in the symbol set
def how_many_symbol(lst):
    return sum(1 for elem in lst if elem in symbol_set)

corpus['how_many_symbol'] = corpus['lexical_morph'].apply(how_many_symbol)

corpus

,norm_text,sex,age,speechAct,tokens,lexical_morph,how_many_symbol
0,다들 비를 싫어하는 이유는 뭐야 ?,여성,20대,(지시) 질문하기,"[(다, MAG), (들, XSN), (비, NNG), (를, JKO), (싫어하,...","[(다, MAG), (비, NNG), (싫어하, VV), (이유, NNG), (뭐,...",3
1,나는 비 좋아 냄새가,여성,20대,(단언) 주장하기,"[(나, NP), (는, JX), (비, NNG), (좋, VA), (어, EC),...","[(나, NP), (비, NNG), (좋, VA), (냄새, NNG)]",3
2,나도 비 좋아하는데 그 소리가,남성,30대,(단언) 주장하기,"[(나, NP), (도, JX), (비, NNG), (좋, VA), (어, EC),...","[(나, NP), (비, NNG), (좋, VA), (그, MM), (소리, NNG)]",5
3,비가 오면 습하고 불쾌지수가 높아져,여성,20대,(단언) 주장하기,"[(비, NNG), (가, JKS), (오, VV), (면, EC), (습, NNG...","[(비, NNG), (오, VV), (습, NNG), (불쾌지수, NNG), (높아...",3
4,맞아 빨래도 안 말라서 걱정,여성,20대,(단언) 주장하기,"[(맞, VV), (어, EC), (빨래, NNG), (도, JX), (안, MAG...","[(맞, VV), (빨래, NNG), (안, MAG), (마르, VV), (걱정, ...",5
...,...,...,...,...,...,...,...
1445938,래브라도 리트리버도 정말 귀엽더라,여성,30대,(단언) 진술하기,"[(래브라도, NNP), (리트리버, NNP), (도, JX), (정말, MAG),...","[(래브라도, NNP), (리트리버, NNP), (정말, MAG), (귀엽, VA-I)]",2
1445939,밀란아~ 하고 이름 부르면 절대 안오거든?,여성,30대,(단언) 진술하기,"[(밀란, NNP), (아, JKV), (~, SO), (하, VV), (고, EC...","[(밀란, NNP), (하, VV), (이름, NNG), (부르, VV), (절대,...",5
1445940,이름 불러도 안온다고?,여성,30대,(지시) 질문하기,"[(이름, NNG), (부르, VV), (어도, EC), (안, MAG), (오, ...","[(이름, NNG), (부르, VV), (안, MAG), (오, VV)]",4
1445941,응 그러다가 아! 맛있다~ 이러면 와,여성,30대,(단언) 진술하기,"[(응, IC), (그러, VV), (다가, EC), (아, IC), (!, SF)...","[(응, IC), (그러, VV), (아, IC), (맛있, VA), (이렇, VA...",3


In [69]:
def compute_ratio(row):
    if len(row['lexical_morph'])!=0:
        ratio = (row['how_many_symbol'] / len(row['lexical_morph'])) * 100
        return f"{ratio:.2f}"
    else:
        return 0

corpus['symbol_ratio'] = corpus.apply(compute_ratio, axis=1)

In [70]:
corpus

,norm_text,sex,age,speechAct,tokens,lexical_morph,how_many_symbol,symbol_ratio
0,다들 비를 싫어하는 이유는 뭐야 ?,여성,20대,(지시) 질문하기,"[(다, MAG), (들, XSN), (비, NNG), (를, JKO), (싫어하,...","[(다, MAG), (비, NNG), (싫어하, VV), (이유, NNG), (뭐,...",3,60.00
1,나는 비 좋아 냄새가,여성,20대,(단언) 주장하기,"[(나, NP), (는, JX), (비, NNG), (좋, VA), (어, EC),...","[(나, NP), (비, NNG), (좋, VA), (냄새, NNG)]",3,75.00
2,나도 비 좋아하는데 그 소리가,남성,30대,(단언) 주장하기,"[(나, NP), (도, JX), (비, NNG), (좋, VA), (어, EC),...","[(나, NP), (비, NNG), (좋, VA), (그, MM), (소리, NNG)]",5,100.00
3,비가 오면 습하고 불쾌지수가 높아져,여성,20대,(단언) 주장하기,"[(비, NNG), (가, JKS), (오, VV), (면, EC), (습, NNG...","[(비, NNG), (오, VV), (습, NNG), (불쾌지수, NNG), (높아...",3,60.00
4,맞아 빨래도 안 말라서 걱정,여성,20대,(단언) 주장하기,"[(맞, VV), (어, EC), (빨래, NNG), (도, JX), (안, MAG...","[(맞, VV), (빨래, NNG), (안, MAG), (마르, VV), (걱정, ...",5,100.00
...,...,...,...,...,...,...,...,...
1445938,래브라도 리트리버도 정말 귀엽더라,여성,30대,(단언) 진술하기,"[(래브라도, NNP), (리트리버, NNP), (도, JX), (정말, MAG),...","[(래브라도, NNP), (리트리버, NNP), (정말, MAG), (귀엽, VA-I)]",2,50.00
1445939,밀란아~ 하고 이름 부르면 절대 안오거든?,여성,30대,(단언) 진술하기,"[(밀란, NNP), (아, JKV), (~, SO), (하, VV), (고, EC...","[(밀란, NNP), (하, VV), (이름, NNG), (부르, VV), (절대,...",5,71.43
1445940,이름 불러도 안온다고?,여성,30대,(지시) 질문하기,"[(이름, NNG), (부르, VV), (어도, EC), (안, MAG), (오, ...","[(이름, NNG), (부르, VV), (안, MAG), (오, VV)]",4,100.00
1445941,응 그러다가 아! 맛있다~ 이러면 와,여성,30대,(단언) 진술하기,"[(응, IC), (그러, VV), (다가, EC), (아, IC), (!, SF)...","[(응, IC), (그러, VV), (아, IC), (맛있, VA), (이렇, VA...",3,50.00


In [71]:
corpus['symbol_ratio'] = pd.to_numeric(corpus['symbol_ratio'], errors='coerce')

symbol_ratio_mean = corpus['symbol_ratio'].mean()

print(symbol_ratio_mean)

59.353808172244676


# 5. 상징이 포함된 문장은 어떤 유형일지?

In [72]:
all_speech_act = corpus.groupby('speechAct').size()

all_speech_act = all_speech_act.sort_values(ascending=False)
all_speech_act

speechAct
(단언) 주장하기                   675273
(단언) 진술하기                   398599
(지시) 질문하기                   268550
(지시) 충고/제안하기                 39538
(언약) 약속하기(제3자와)/(개인적 수준)     17959
턴토크 사인(관습적 반응)                8795
(표현) 긍정감정 표현하기                7560
(표현) 부정감정 표현하기                7319
(지시) 부탁하기                     5674
N/A                           4093
(지시) 명령/요구하기                  3909
(단언) 반박하기                     3513
(표현) 감사하기                     2491
(표현) 인사하기                     1163
(표현) 사과하기                      791
(언약) 거절하기                      524
(선언/위임하기)                      119
(언약) 위협하기                       72
(언약) 약속하기                        1
dtype: int64

In [73]:
# 상징이 사용된 문장이라면

# Filter rows where 'how_many_symbol' is not 0
corpus_with_symbol = corpus[corpus['how_many_symbol'] != 0]

# Group by 'speechAct' and count the occurrences of each category
speech_act_counts = corpus_with_symbol.groupby('speechAct').size()

speech_act_counts = speech_act_counts.sort_values(ascending=False)
speech_act_counts

speechAct
(단언) 주장하기                   639015
(단언) 진술하기                   384321
(지시) 질문하기                   254300
(지시) 충고/제안하기                 38665
(언약) 약속하기(제3자와)/(개인적 수준)     17535
(표현) 부정감정 표현하기                6701
(표현) 긍정감정 표현하기                6529
(지시) 부탁하기                     5346
턴토크 사인(관습적 반응)                5159
N/A                           3727
(지시) 명령/요구하기                  3683
(단언) 반박하기                     2878
(표현) 감사하기                     2405
(표현) 인사하기                      817
(표현) 사과하기                      618
(언약) 거절하기                      472
(선언/위임하기)                       93
(언약) 위협하기                       69
(언약) 약속하기                        1
dtype: int64

In [74]:
# 상징이 안 사용된 문장은?

# Filter rows where 'how_many_symbol' is 0
corpus_without_symbol = corpus[corpus['how_many_symbol'] == 0]

no_symbol_speech_act_counts = corpus_without_symbol.groupby('speechAct').size()

no_symbol_speech_act_counts = no_symbol_speech_act_counts.sort_values(ascending=False)
no_symbol_speech_act_counts

speechAct
(단언) 주장하기                   36258
(단언) 진술하기                   14278
(지시) 질문하기                   14250
턴토크 사인(관습적 반응)               3636
(표현) 긍정감정 표현하기               1031
(지시) 충고/제안하기                  873
(단언) 반박하기                     635
(표현) 부정감정 표현하기                618
(언약) 약속하기(제3자와)/(개인적 수준)      424
N/A                           366
(표현) 인사하기                     346
(지시) 부탁하기                     328
(지시) 명령/요구하기                  226
(표현) 사과하기                     173
(표현) 감사하기                      86
(언약) 거절하기                      52
(선언/위임하기)                      26
(언약) 위협하기                       3
dtype: int64

# 6. 성별 및 나이에 따라 상징 활용에 차이가 있는지?

In [75]:
corpus.groupby('sex').size()

sex
남성    559906
여성    886037
dtype: int64

In [76]:
corpus.groupby('age').size()

age
10대        29091
20대       707979
30대       544227
40대       114594
50대        27081
60대 이상     22971
dtype: int64

In [77]:
grouped_df = {}

sex_age = corpus.groupby(['sex', 'age'])

# Loop through the groups and create separate DataFrames
for (sex, age), group in sex_age:
    group_name = f"sex_{sex}_age_{age}"
    grouped_df[group_name] = group
    
grouped_df['sex_여성_age_10대']

,norm_text,sex,age,speechAct,tokens,lexical_morph,how_many_symbol,symbol_ratio
333,너는 연말이나 새해에 파티를 한다면 어떤 음식 먹고 싶어?,여성,10대,(지시) 질문하기,"[(너, NP), (는, JX), (연말, NNG), (이나, JC), (새해, N...","[(너, NP), (연말, NNG), (새해, NNG), (파티, NNG), (하,...",6,75.00
334,나는 일단 케이크 맛있고 비싼거야 해,여성,10대,(단언) 주장하기,"[(나, NP), (는, JX), (일단, MAG), (케이크, NNG), (맛있,...","[(나, NP), (일단, MAG), (케이크, NNG), (맛있, VA), (비싸...",6,85.71
335,그리고 폭립같은 정석적인 파티 음식도 좋고 그냥 돼지고기 구워먹어도 좋아,여성,10대,(단언) 주장하기,"[(그리고, MAJ), (폭, NNG), (립, NNG), (같, VA), (은, ...","[(그리고, MAJ), (폭, NNG), (립, NNG), (같, VA), (정석,...",11,84.62
336,넌 새해에 파티한다면 뭐 먹고 싶어?,여성,10대,(지시) 질문하기,"[(너, NP), (ᆫ, JX), (새해, NNG), (에, JKB), (파티, N...","[(너, NP), (새해, NNG), (파티, NNG), (뭐, NP), (먹, VV)]",4,80.00
337,오 나도 우선은 케이크!,여성,10대,(단언) 주장하기,"[(오, IC), (나, NP), (도, JX), (우선, NNG), (은, JX)...","[(오, IC), (나, NP), (우선, NNG), (케이크, NNG)]",2,50.00
...,...,...,...,...,...,...,...,...
1445335,너네도 자세교정 하는게 좋지 않아?,여성,10대,(지시) 질문하기,"[(너, NP), (네, XSN), (도, JX), (자세, NNG), (교정, N...","[(너, NP), (자세, NNG), (교정, NNG), (하, VV), (것, N...",4,66.67
1445338,너네 볼때마다 목 뒤가 굽어져있던데 그래도 할 필요가 없어?,여성,10대,(지시) 질문하기,"[(너, NP), (네, XSN), (보, VV), (ᆯ, ETM), (때, NNG...","[(너, NP), (보, VV), (때, NNG), (목, NNG), (뒤, NNG...",7,70.00
1445809,엄청 잔인하던데.,여성,10대,(단언) 주장하기,"[(엄청, MAG), (잔인, NNG), (하, XSA), (던데, EF), (.,...","[(엄청, MAG), (잔인, NNG)]",0,0.00
1445812,무슨 일인데?,여성,10대,(지시) 질문하기,"[(무슨, MM), (일, NNG), (이, VCP), (ᆫ데, EF), (?, SF)]","[(무슨, MM), (일, NNG)]",1,50.00


In [78]:
# 어떤 품사의 형태소가 몇 개 쓰였는지

def count_pos_symbol(lexical_morph, my_set):
    count_dict = {}
    
    for lst in lexical_morph:
        for tup in lst:
            if tup in my_set:
                pos = tup[1]
                
                if pos in count_dict:
                    count_dict[pos] += 1
                else:
                    count_dict[pos] = 1
    
    count_dict = dict(sorted(count_dict.items(), key=lambda item: item[1], reverse=True))
                    
    return count_dict

In [79]:
f_10 = grouped_df['sex_여성_age_10대']

print(f_10['symbol_ratio'].mean())
print("-----------------------")
count_pos_symbol(f_10['lexical_morph'], symbol_set)

60.917287830541326
-----------------------


{'NNG': 8827,
 'VV': 6122,
 'VA': 3051,
 'MAG': 2725,
 'NP': 2073,
 'NNB': 1613,
 'MM': 454,
 'VA-I': 445,
 'IC': 293,
 'NNP': 253,
 'VV-R': 184,
 'NR': 158,
 'VV-I': 148,
 'MAJ': 61,
 'XR': 27}

In [80]:
f_20 = grouped_df['sex_여성_age_20대']

print(f_20['symbol_ratio'].mean())
print("-----------------------")
count_pos_symbol(f_20['lexical_morph'], symbol_set)

59.350581091363125
-----------------------


{'NNG': 491640,
 'VV': 335308,
 'MAG': 175973,
 'VA': 146773,
 'NP': 96314,
 'NNB': 81504,
 'IC': 33525,
 'MM': 27606,
 'VA-I': 23093,
 'NNP': 13345,
 'VV-R': 11701,
 'NR': 10587,
 'VV-I': 6052,
 'MAJ': 2918,
 'XR': 988,
 'VA-R': 155}

In [81]:
f_30 = grouped_df['sex_여성_age_30대']

print(f_30['symbol_ratio'].mean())
print("-----------------------")
count_pos_symbol(f_30['lexical_morph'], symbol_set)

58.602289009316856
-----------------------


{'NNG': 446449,
 'VV': 290940,
 'MAG': 147465,
 'VA': 126163,
 'NNB': 71360,
 'NP': 65927,
 'IC': 28735,
 'MM': 24203,
 'VA-I': 18966,
 'NNP': 12152,
 'NR': 10390,
 'VV-R': 9583,
 'VV-I': 4764,
 'MAJ': 1380,
 'XR': 928,
 'VA-R': 159}

In [82]:
f_40 = grouped_df['sex_여성_age_40대']

print(f_40['symbol_ratio'].mean())
print("-----------------------")
count_pos_symbol(f_40['lexical_morph'], symbol_set)

60.976995223058864
-----------------------


{'NNG': 79558,
 'VV': 50728,
 'MAG': 25242,
 'VA': 23767,
 'NNB': 11975,
 'NP': 9859,
 'IC': 6579,
 'MM': 4555,
 'VA-I': 3480,
 'NNP': 2178,
 'NR': 1833,
 'VV-R': 1814,
 'VV-I': 857,
 'XR': 173,
 'MAJ': 169,
 'VA-R': 37}

In [83]:
f_50 = grouped_df['sex_여성_age_50대']

print(f_50['symbol_ratio'].mean())
print("-----------------------")
count_pos_symbol(f_50['lexical_morph'], symbol_set)

62.0891297155723
-----------------------


{'NNG': 22666,
 'VV': 15950,
 'VA': 8303,
 'MAG': 6769,
 'NNB': 3739,
 'NP': 2825,
 'IC': 2050,
 'MM': 1217,
 'VA-I': 1143,
 'NNP': 595,
 'NR': 422,
 'VV-R': 362,
 'VV-I': 304,
 'MAJ': 130,
 'XR': 72,
 'VA-R': 12}

In [84]:
f_60 = grouped_df['sex_여성_age_60대 이상']

print(f_60['symbol_ratio'].mean())
print("-----------------------")
count_pos_symbol(f_60['lexical_morph'], symbol_set)

62.25420845070422
-----------------------


{'NNG': 12574,
 'VV': 9436,
 'MAG': 4247,
 'VA': 4064,
 'NNB': 1966,
 'IC': 1110,
 'NP': 1038,
 'MM': 669,
 'VA-I': 487,
 'NR': 310,
 'VV-R': 244,
 'NNP': 218,
 'VV-I': 162,
 'XR': 37,
 'MAJ': 29,
 'VA-R': 9}

In [85]:
m_10 = grouped_df['sex_남성_age_10대']

print(m_10['symbol_ratio'].mean())
print("-----------------------")
count_pos_symbol(m_10['lexical_morph'], symbol_set)

65.66423175416134
-----------------------


{'NNG': 24928,
 'VV': 17147,
 'VA': 8569,
 'MAG': 8430,
 'NP': 6394,
 'NNB': 4912,
 'VA-I': 1397,
 'MM': 1210,
 'IC': 1168,
 'NNP': 538,
 'VV-R': 523,
 'VV-I': 522,
 'NR': 302,
 'MAJ': 137,
 'XR': 62,
 'VA-R': 5}

In [86]:
m_20 = grouped_df['sex_남성_age_20대']

print(m_20['symbol_ratio'].mean())
print("-----------------------")
count_pos_symbol(m_20['lexical_morph'], symbol_set)

59.089357619250606
-----------------------


{'NNG': 310558,
 'VV': 217301,
 'MAG': 118385,
 'VA': 97583,
 'NP': 63822,
 'NNB': 56535,
 'IC': 27338,
 'MM': 18470,
 'VA-I': 15915,
 'NNP': 9347,
 'VV-R': 6977,
 'NR': 6355,
 'VV-I': 3848,
 'MAJ': 1033,
 'XR': 678,
 'VA-R': 91}

In [87]:
m_30 = grouped_df['sex_남성_age_30대']

print(m_30['symbol_ratio'].mean())
print("-----------------------")
count_pos_symbol(m_30['lexical_morph'], symbol_set)

59.39780808432124
-----------------------


{'NNG': 177407,
 'VV': 122512,
 'MAG': 61883,
 'VA': 53159,
 'NNB': 29820,
 'NP': 28812,
 'IC': 20156,
 'MM': 10751,
 'VA-I': 8560,
 'NNP': 5568,
 'VV-R': 4165,
 'NR': 3923,
 'VV-I': 1887,
 'MAJ': 420,
 'XR': 406,
 'VA-R': 60}

In [88]:
m_40 = grouped_df['sex_남성_age_40대']

print(m_40['symbol_ratio'].mean())
print("-----------------------")
count_pos_symbol(m_40['lexical_morph'], symbol_set)

58.50571604307424
-----------------------


{'NNG': 60225,
 'VV': 40703,
 'MAG': 18980,
 'VA': 17124,
 'NNB': 11559,
 'NP': 8184,
 'IC': 6004,
 'MM': 3654,
 'NNP': 2760,
 'VA-I': 2488,
 'VV-R': 1517,
 'NR': 1304,
 'VV-I': 654,
 'MAJ': 210,
 'XR': 137,
 'VA-R': 17}

In [89]:
m_50 = grouped_df['sex_남성_age_50대']

print(m_50['symbol_ratio'].mean())
print("-----------------------")
count_pos_symbol(m_50['lexical_morph'], symbol_set)

65.88434749905979
-----------------------


{'NNG': 7808,
 'VV': 5426,
 'MAG': 2255,
 'VA': 2180,
 'NNB': 1359,
 'NP': 923,
 'MM': 468,
 'VA-I': 283,
 'NR': 227,
 'VV-R': 162,
 'NNP': 130,
 'IC': 118,
 'VV-I': 61,
 'XR': 19,
 'MAJ': 12,
 'VA-R': 2}

In [90]:
m_60 = grouped_df['sex_남성_age_60대 이상']

print(m_60['symbol_ratio'].mean())
print("-----------------------")
count_pos_symbol(m_60['lexical_morph'], symbol_set)

61.32542813083354
-----------------------


{'NNG': 14046,
 'VV': 8314,
 'VA': 4138,
 'MAG': 3392,
 'NNB': 1705,
 'NP': 939,
 'IC': 899,
 'MM': 685,
 'VA-I': 661,
 'NNP': 414,
 'NR': 292,
 'VV-R': 260,
 'VV-I': 170,
 'XR': 25,
 'MAJ': 19,
 'VA-R': 2}